In [7]:
import tensorflow as tf
import sys, os
sys.path.append(os.path.abspath('..'))
from csp.utils import utils

In [34]:
hparams = tf.contrib.training.HParams(
    name="attention_aps_word",
    num_units=320,
    model="attention",
    dataset="aps",
    input_unit="word",
    out_dir=None,
    beam_width=0,
)
hparams.out_dir = "../saved_models/%s" % hparams.name
mode = tf.estimator.ModeKeys.PREDICT

BatchedInput = utils.get_batched_input_class(hparams)
Model = utils.get_model_class(hparams)

In [35]:
def load_input(self, filename):
    fh = open(filename, "rb")
    spam = fh.read(12)
    nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
    veclen = int(sampSize / 4)
    fh.seek(12, 0)
    dat = np.fromfile(fh, dtype=np.float32)
    if len(dat) % veclen != 0: dat = dat[:len(dat) - len(dat) % veclen]
    dat = dat.reshape(len(dat) // veclen, veclen)
    dat = dat.byteswap()
    fh.close()
    return dat

In [36]:
graph = tf.Graph()
filenames = tf.placeholder(dtype=tf.string)
dataset = tf.data.Dataset.from_tensor_slices(filenames)
dataset = dataset.map(lambda filename: tf.py_func(load_input, [filename], tf.float32))
datast = dataset.map(lambda feat: (feat, tf.shape(feat)[0]))
iterator = dataset.make_initializable_iterator()
hparams.num_classes = 34331
with graph.as_default():
    model = Model(
        hparams,
        mode=mode,
        iterator=iterator
    )
    sess = tf.Session(graph=graph)
    latest_ckpt = tf.train.lastest_checkpoint(hparams.out_dir)
    model.saver.restore(sess, latest_ckpt)
    global_step = model.global_step.eval(session=sess)
    # reset iterator
    

TypeError: Tensor objects are not iterable when eager execution is not enabled. To iterate over this tensor use tf.map_fn.